In [ ]:
import ipyrad.analysis as ipa
import toytree
import toyplot
import toyplot.pdf

In [ ]:
# Setup the path to your HDF5 file
data = "/staging/leuven/stg_00026/enora/trem_ipyrad/new_trem/trem_default_popfile_outfiles/trem_default_popfile.snps.hdf5"

In [ ]:
# Group individuals into populations
imap = {
    "pop1": ["ind1", "ind2"],
    "pop2": ["ind3", "ind4", "ind5", "ind6", "ind7", "ind8"],
    "pop3": ["ind9", "ind10", "ind11", "ind12", "ind13"],
    "pop4": ["ind14", "ind15", "ind16", "ind17", "ind18", "ind19", "ind20"],
}

# Minimum % of samples that must be present in each SNP from each group
minmap = {i: 0.5 for i in imap}

In [ ]:
# initiate a Treemix analysis object with some parameters arguments
tmx = ipa.treemix(
    data=data,
    imap=imap,
    minmap=minmap,
    seed=123456,
    root="pop1",
    m=2,
)

In [ ]:
# Run Treemix analysis
tmx.run()

# Draw resulting tree
tmx.draw_tree();

In [ ]:
# Draw covariance matrix
tmx.draw_cov();

In [ ]:
# Initiate new Treemix analysis to find the best value for m
tmx = ipa.treemix(
    data=data,
    imap=imap,
    minmap=minmap,
    seed=1234,
    root="pop1",
)

In [ ]:
# Iterate your run with different values of m
tests = {}
nadmix = [0, 1, 2, 3, 4, 5, 6]

for adm in nadmix:
    tmx.params.m = adm
    tmx.run()
    tests[adm] = tmx.results.llik

In [ ]:
# Plot the likelihood for the different values of m
toyplot.plot(
    nadmix,
    [tests[i] for i in nadmix],
    width=350,
    height=275,
    stroke_width=3,
    xlabel="n admixture edges",
    ylabel="ln(likelihood)",
);

In [ ]:
# Initiate a gridded canvas to plot trees on
canvas = toyplot.Canvas(width=600, height=700)

# Iterate on different set of random SNPs (9 here)
for i in range(9):

    # Initiate a treemix analysis object with a random seed
    tmx = ipa.treemix(
        data=data,
        imap=imap,
        minmap=minmap,
        root="pop1",
        global_=True,
        m=3,
        quiet=True
    )

    # Run model fit
    tmx.run()

    # Select a plot grid axis and add tree to axes
    axes = canvas.cartesian(grid=(3, 3, i))
    tmx.draw_tree(axes)

In [ ]:
# Save your plot as PDF file
toyplot.pdf.render(canvas, "treemix-trem_default_m3.pdf")